# Objetivo do Código de Recomendação de Jogos
---
O objetivo deste código é fornecer recomendações de jogos com base em um conjunto de dados de avaliações de jogos. Ele utiliza algoritmos de filtragem e pontuação para selecionar os jogos mais relevantes de acordo com critérios fornecidos, como título, gênero, preço, data de lançamento e plataforma.


---

*A sequência de execução do código é a seguinte:*

* Carregar os dados do conjunto de dados disponível em um URL.
* Realizar o pré-processamento dos dados, convertendo as colunas de avaliações para tipo numérico e removendo as linhas com valores ausentes nas colunas de avaliações.
* Calcular a média das avaliações positivas dos jogos em todo o conjunto de dados.
* Calcular a pontuação de cada jogo com base nas avaliações positivas e negativas, a média das avaliações positivas e um valor de referência mínimo necessário para um jogo atender ao pré-requisito.
* Filtrar os jogos que atendem ao pré-requisito de ter uma quantidade mínima de avaliações positivas.
* Receber os parâmetros de busca, como título, gênero, preço, data de lançamento e plataforma, e retornar um DataFrame com as principais recomendações de jogos que atendem aos critérios especificados, ordenados por pontuação.


# Algoritmo

## Carrega o conjunto

In [42]:
from datetime import datetime
import pandas as pd

def load_data():
    url = 'https://drive.google.com/uc?export=download&id=1SdCK7a5E9_vQKTEmMtEROrq-LBsKlTPE'
    data = pd.read_csv(url, dtype={'id': str}, low_memory=False)
    return data

## Pré-processamento

In [43]:
def preprocess_data(data):
    data['positive_ratings'] = pd.to_numeric(data['positive_ratings'], errors='coerce')
    data['negative_ratings'] = pd.to_numeric(data['negative_ratings'], errors='coerce')
    data.dropna(subset=['positive_ratings', 'negative_ratings'], inplace=True)
    return data

## Calcula média

In [44]:
def calculate_mean_rating(data):
    mean_rating = data['positive_ratings'].mean()
    return mean_rating

## Calcular pontuação

In [45]:
def calculate_score(row, mean_rating, m):
    v = row['positive_ratings']
    r = row['negative_ratings']
    return (v / (v + m) * r) + (m / (m + v) * mean_rating)

##  Filtra para o pré-requisito

In [46]:
def filter_games(data, m):
    filtered_games = data[data['positive_ratings'] >= m].copy()
    return filtered_games

## Obtem recomendação

In [47]:
def get_game_recommendations(data, filtered_games, title=None, genre=None, top_n=50, price=None, release_date=None, platforms=None):
    games = filtered_games.copy()

    if title:
        games = games[games['name'].str.contains(title, case=False)]
    if genre:
        games = games[games['genres'].str.contains(genre, case=False)]
    if price is not None:
        games = games[games['price'] <= price]
    if release_date is not None:
        release_date = datetime.strptime(release_date, '%Y-%m-%d').date()
        games = games[pd.to_datetime(games['release_date']).dt.date >= release_date]
    if platforms:
        games = games[games['platforms'].str.contains(platforms, case=False)]

    games = games.sort_values('score', ascending=False)
    recommendations = games[['name', 'price', 'release_date', 'platforms', 'genres', 'positive_ratings', 'negative_ratings', 'score']].head(top_n)
    styled_recommendations = recommendations.style.set_properties(**{'text-align': 'left'})
    return styled_recommendations

# Execução

In [48]:
data = load_data()
data = preprocess_data(data)
m = data['positive_ratings'].quantile(0.70)
filtered_games = filter_games(data, m)
mean_rating = calculate_mean_rating(data)
filtered_games['score'] = filtered_games.apply(calculate_score, args=(mean_rating, m), axis=1)

## Testes

In [49]:
recommendations = get_game_recommendations(data, filtered_games, title='Action', top_n=10)
display(recommendations)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
3509,Naval Action,29.990000,2016-01-21,windows,Action;Adventure;Indie;Massively Multiplayer;Simulation;Strategy;Early Access,3117,2029,2002.324799
14503,SoulWorker - Anime Action MMO,0.000000,2018-02-26,windows,Action;Free to Play;Massively Multiplayer;RPG,5953,1904,1891.576931
3742,Enforcer: Police Crime Action,14.990000,2014-10-24,windows;mac,Action;Adventure;Simulation,1028,1119,1110.151537
928,Red Faction®: Armageddon™,14.990000,2011-06-09,windows,Action,2674,1005,1004.866288
3719,Double Action: Boogaloo,0.000000,2014-10-23,windows;linux,Action;Free to Play;Indie,9252,786,787.907698
7341,Tempest: Pirate Action RPG,14.990000,2016-08-22,windows;mac,Action;Indie;RPG,1221,501,532.797053
11206,Fast Action Hero,8.990000,2016-10-03,windows,Violent;Action;Casual;Indie;Strategy;Early Access,83,11,505.779261
8229,Smash Up: Conquer the bases with your factions,5.190000,2016-10-12,windows,Strategy,102,75,490.250580
5177,No Turning Back: The Pixel Art Action-Adventure Roguelike,1.990000,2016-01-15,windows,Action;Adventure;Indie;RPG;Early Access,116,94,472.112349
14051,Runic Rampage - Action RPG,4.990000,2017-05-17,windows;mac;linux,Action;Adventure;Indie;RPG,97,13,468.374208


In [50]:
recommendations = get_game_recommendations(data, filtered_games, title='War', top_n=10)
display(recommendations)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
1727,War Thunder,0.000000,2013-08-15,windows;mac;linux,Action;Free to Play;Massively Multiplayer;Simulation,83852,22166,22145.070332
1634,Warframe,0.000000,2013-03-25,windows,Action;Free to Play,226541,20268,20260.943388
2951,Warface,0.000000,2014-07-01,windows,Action;Free to Play;Massively Multiplayer,32751,14409,14375.105237
1421,Total War™: ROME II - Emperor Edition,29.990000,2013-09-02,windows;mac,Strategy,31864,13423,13390.725838
1479,Chivalry: Medieval Warfare,18.990000,2012-10-16,windows;mac;linux,Action;Indie,51499,10939,10923.008169
2991,Call of Duty®: Infinite Warfare,39.990000,2016-11-03,windows,Action;Adventure,7821,10570,10469.511179
5404,Total War: WARHAMMER,39.990000,2016-05-24,windows;mac;linux,Action;Strategy,29826,9878,9853.364350
1346,Call of Duty®: Advanced Warfare - Gold Edition,39.990000,2014-11-03,windows,Action,13389,8379,8333.541965
2775,"Warhammer 40,000: Dawn of War III",29.990000,2017-04-27,windows;mac;linux,Violent;Gore;Action;Simulation;Strategy,6793,8336,8247.454095
11867,Warhammer: Vermintide 2,23.790000,2018-03-08,windows,Action,21509,7336,7311.646460


In [51]:
recommendations = get_game_recommendations(data, filtered_games, platforms='Windows', top_n=10)
display(recommendations)

,name,price,release_date,platforms,genres,positive_ratings,negative_ratings,score
12836,PLAYERUNKNOWN'S BATTLEGROUNDS,26.990000,2017-12-21,windows,Action;Adventure;Massively Multiplayer,496184,487076,486994.704525
25,Counter-Strike: Global Offensive,0.000000,2012-08-21,windows;mac;linux,Action;Free to Play,2644404,402313,402300.404388
22,Dota 2,0.000000,2013-07-09,windows;mac;linux,Action;Free to Play;Strategy,863507,142079,142065.440891
2478,Grand Theft Auto V,24.990000,2015-04-13,windows,Action;Adventure,329061,139308,139273.123114
7809,Z1 Battle Royale,0.000000,2018-02-28,windows,Action;Adventure;Free to Play;Massively Multiplayer,111434,91664,91596.520910
1506,DayZ,33.990000,2018-12-13,windows,Action;Adventure;Massively Multiplayer,118435,77169,77115.658055
4712,ARK: Survival Evolved,44.990000,2017-08-27,windows;mac;linux,Action;Adventure;Indie;Massively Multiplayer;RPG,145035,66603,66565.478792
5235,Tom Clancy's Rainbow Six® Siege,16.990000,2015-12-01,windows,Action,251178,59620,59600.636017
1467,PAYDAY 2,7.490000,2013-08-13,windows;linux,Action;RPG,308657,56523,56508.073646
2563,No Man's Sky,39.990000,2016-08-12,windows,Action;Adventure,63444,56488,56415.503933
